In [1]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.transform import Rotation

In [2]:
def read_coords():
    coords_dict = {}
    model_count = 0
    with open("1fsd.pdb",'r') as f:
        for line in f:
            tmp_line = line.split()
            if tmp_line[0] == "MODEL":
                model_count += 1
                coords_dict[model_count] = []

            elif tmp_line[0]  == 'ATOM' and tmp_line[2] == 'CA':
                coords = tmp_line[6:9]
                coords_dict[model_count].append(coords)

    return coords_dict

In [3]:
def is_same_matrix(matrix, composed_matrix):
    return np.allclose(matrix, composed_matrix)

In [4]:
# return small matrix (5, 28)
def build_small_matrix(matrix):

    q, r = np.linalg.qr(matrix)
    inds = np.where(np.where(abs(r) > 0.001, 1, 0).any(axis=1))[0]
    return matrix[inds, :], inds

In [5]:
# восстановление первоначальной матрицы порядка 28
def compose_matrix(small_matrix, independent_rows_inds):

    composed_matrix = np.zeros((28,28))
    dependent_rows_inds = np.setdiff1d(np.arange(28), independent_rows_inds)
    for i in dependent_rows_inds:
        coef = np.linalg.solve(small_matrix[:, :5].T, small_matrix[:, i])
        composed_matrix[i, :] = np.sum(small_matrix * coef[:, np.newaxis], axis=0)

    for ind_number, ind in enumerate(independent_rows_inds):
        composed_matrix[ind, :] = small_matrix[ind_number]

    return composed_matrix

In [6]:
def transform(matrix):

    movement_vector = np.array([1, 5, 2])
    moved_vector = np.asarray(matrix, dtype=float) + movement_vector

    x_angle = 0
    y_angle = 0
    z_angle = 90

    r = Rotation.from_euler('zxy', [[x_angle, y_angle, z_angle]], degrees=True)
    moved_matrix = r.apply(moved_vector)
    return moved_matrix

In [7]:
def compute_model(model_coords):
    matrix = euclidean_distances(model_coords, model_coords) ** 2
    print('rank =', np.linalg.matrix_rank(matrix))

    small_matrix, inds = build_small_matrix(matrix)
    print('shape of small matrix is =', small_matrix.shape)

    composed_matrix = compose_matrix(small_matrix, inds)
    print('matrix and composed matrix are the same?', is_same_matrix(matrix, composed_matrix))
    return composed_matrix

In [10]:
coords_dict = read_coords()
for model_id in range(1, 42):
    print('MODEL', model_id, end=', ')
    composed_matrix = compute_model(coords_dict[model_id])

    # подвинули, повернули точки
    moved_model = transform(coords_dict[model_id])
    composed_moved_matrix = compute_model(moved_model)

    print('Are matrix and matrix for moved points the same?',np.allclose(composed_matrix, composed_moved_matrix))

MODEL 1, rank = 5
shape of small matrix is = (5, 28)
matrix and composed matrix are the same? True
rank = 5
shape of small matrix is = (5, 28)
matrix and composed matrix are the same? True
Are matrix and matrix for moved points the same? True
MODEL 2, rank = 5
shape of small matrix is = (5, 28)
matrix and composed matrix are the same? True
rank = 5
shape of small matrix is = (5, 28)
matrix and composed matrix are the same? True
Are matrix and matrix for moved points the same? True
MODEL 3, rank = 5
shape of small matrix is = (5, 28)
matrix and composed matrix are the same? True
rank = 5
shape of small matrix is = (5, 28)
matrix and composed matrix are the same? True
Are matrix and matrix for moved points the same? True
MODEL 4, rank = 5
shape of small matrix is = (5, 28)
matrix and composed matrix are the same? True
rank = 5
shape of small matrix is = (5, 28)
matrix and composed matrix are the same? True
Are matrix and matrix for moved points the same? True
MODEL 5, rank = 5
shape of s

In [ ]:
Task №3

По теореме 3.8 ранг матрицы может быть максимум 5 = r + 2, где r - embedding dimension = 3; 
Также ранг матрицы, где точек больше 3, не может быть меньше 4, по той же теореме.
https://books.google.ru/books?id=woJyDwAAQBAJ&pg=PA63&lpg=PA63&dq=Theorem+rank+of+EDM+proof&source=bl&ots=GT-j10AfbE&sig=ACfU3U0GFPAk4s3PDtV0t0IDUV4ejuhmUA&hl=ru&sa=X&ved=2ahUKEwjZxvDX8oLkAhXKxcQBHXTHC3oQ6AEwFXoECAcQAQ#v=snippet&q=Gower&f=false

Embedding dimension, так как точки у нас находятся в general position, равна 3.

По теореме 1.2 ранг матрицы будет равен 4 только и только тогда, когда точки лежат на 3-гиперсфере.
https://infoscience.epfl.ch/record/196439/files/EPFL_TH5971.pdf 

Точки для всех моделей не лежат на гиперсфере, значит ранги всех матриц будут равны 5.